In [1]:
import os
import numpy as np
import pandas as pd

from quantifier_recommender import QuantifierRecommender

quantifier_recommender = QuantifierRecommender()

quantifier_recommender.construct_meta_table(datasets_path="./datasets/",
                                            train_data_path="./data/train_data/",
                                            test_data_path="./data/test_data/",
                                            supervised=True)

quantifier_recommender.evaluation_table.to_csv("./recommender_evaluation_table.csv", index=False)
quantifier_recommender.meta_features_table.to_csv("./recommender_meta_features_table.csv", index=False)

c:\Users\coenL\AppData\Local\Programs\Python\Python312\Lib\site-packages\pymfe\_internal.py:1568: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")
c:\Users\coenL\AppData\Local\Programs\Python\Python312\Lib\site-packages\pymfe\_internal.py:1281: UserWarning:  * Something went wrong while precomputing 'precompute_can_cors'. Will ignore this method. Error message:
TypeError("OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'").
  warnings.warn(


  Fold #1
Training_len 16800
SCORES_Length: 1680
  Fold #2
Training_len 16800
SCORES_Length: 3360
  Fold #3
Training_len 16800
SCORES_Length: 5040
  Fold #4
Training_len 16800
SCORES_Length: 6720
  Fold #5
Training_len 16800
SCORES_Length: 8400
  Fold #6
Training_len 16800
SCORES_Length: 10080
  Fold #7
Training_len 16800
SCORES_Length: 11760
  Fold #8
Training_len 16800
SCORES_Length: 13440
  Fold #9
Training_len 16800
SCORES_Length: 15120
  Fold #10
Training_len 16800
SCORES_Length: 16800


c:\Guizo\quantifier_recommender\utils\getTPRFPR.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  TprFpr = pd.concat([TprFpr, aux], ignore_index=True)
c:\Guizo\quantifier_recommender\quantifiers\ACC.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if (float(TprFpr['tpr']) - float(TprFpr['fpr'])) == 0:
c:\Guizo\quantifier_recommender\quantifiers\ACC.py:31: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pos_prop = (cc_ouput - float(TprFpr['fpr']))/(float(TprFpr['tpr']) - float(TprFpr['fpr']))   #adjusted class proportion
c:\Guizo\quantifier_r

  Fold #1
Training_len 16800
SCORES_Length: 1680
  Fold #2
Training_len 16800
SCORES_Length: 3360
  Fold #3
Training_len 16800
SCORES_Length: 5040
  Fold #4
Training_len 16800
SCORES_Length: 6720
  Fold #5
Training_len 16800
SCORES_Length: 8400
  Fold #6
Training_len 16800
SCORES_Length: 10080
  Fold #7
Training_len 16800
SCORES_Length: 11760
  Fold #8
Training_len 16800
SCORES_Length: 13440
  Fold #9
Training_len 16800
SCORES_Length: 15120
  Fold #10
Training_len 16800
SCORES_Length: 16800


c:\Guizo\quantifier_recommender\utils\getTPRFPR.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  TprFpr = pd.concat([TprFpr, aux], ignore_index=True)
c:\Guizo\quantifier_recommender\quantifiers\ACC.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if (float(TprFpr['tpr']) - float(TprFpr['fpr'])) == 0:
c:\Guizo\quantifier_recommender\quantifiers\ACC.py:31: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pos_prop = (cc_ouput - float(TprFpr['fpr']))/(float(TprFpr['tpr']) - float(TprFpr['fpr']))   #adjusted class proportion
c:\Guizo\quantifier_r

  Fold #1
Training_len 4609
SCORES_Length: 461
  Fold #2
Training_len 4609
SCORES_Length: 922
  Fold #3
Training_len 4609
SCORES_Length: 1383
  Fold #4
Training_len 4609
SCORES_Length: 1844
  Fold #5
Training_len 4609
SCORES_Length: 2305
  Fold #6
Training_len 4609
SCORES_Length: 2766
  Fold #7
Training_len 4609
SCORES_Length: 3227
  Fold #8
Training_len 4609
SCORES_Length: 3688
  Fold #9
Training_len 4609
SCORES_Length: 4149
  Fold #10
Training_len 4609
SCORES_Length: 4609


c:\Guizo\quantifier_recommender\utils\getTPRFPR.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  TprFpr = pd.concat([TprFpr, aux], ignore_index=True)
c:\Guizo\quantifier_recommender\quantifiers\ACC.py:28: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if (float(TprFpr['tpr']) - float(TprFpr['fpr'])) == 0:
c:\Guizo\quantifier_recommender\quantifiers\ACC.py:31: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  pos_prop = (cc_ouput - float(TprFpr['fpr']))/(float(TprFpr['tpr']) - float(TprFpr['fpr']))   #adjusted class proportion
c:\Guizo\quantifier_r